In [38]:
import pandas as pd

In [39]:
schools_gs = pd.read_csv('schools_gs.csv')
print(schools_gs.shape)
schools_gs.head()

(950, 4)


,Unnamed: 0,School,Students,District
0,0,10/10Top ratedGreatSchools’ Summary Rating pro...,808.0,Weston School District
1,1,3/10Below averageGreatSchools’ Summary Rating ...,361.0,West Hartford School District
2,2,4/10Below averageGreatSchools’ Summary Rating ...,331.0,New Haven School District
3,3,4/10Below averageGreatSchools’ Summary Rating ...,603.0,Stamford School District
4,4,7/10Above averageGreatSchools’ Summary Rating ...,484.0,Danbury School District


In [40]:
# Convert the "School" and "District" fields to string and get rid of "Unnamed: 0"
schools_gs = schools_gs.astype(
    {
        'School': str,
        'District': str
    }
).drop(
    'Unnamed: 0',
    axis = 1
)
schools_gs

,School,Students,District
0,10/10Top ratedGreatSchools’ Summary Rating pro...,808.0,Weston School District
1,3/10Below averageGreatSchools’ Summary Rating ...,361.0,West Hartford School District
2,4/10Below averageGreatSchools’ Summary Rating ...,331.0,New Haven School District
3,4/10Below averageGreatSchools’ Summary Rating ...,603.0,Stamford School District
4,7/10Above averageGreatSchools’ Summary Rating ...,484.0,Danbury School District
...,...,...,...
945,6/10AverageGreatSchools’ Summary Rating provid...,933.0,West Hartford School District
946,5/10AverageGreatSchools’ Summary Rating provid...,582.0,Connecticut Technical High School
947,9/10Above averageGreatSchools’ Summary Rating ...,1658.0,Ridgefield School District
948,4/10Below averageGreatSchools’ Summary Rating ...,320.0,Wallingford School District


In [41]:
schools_gs['Rating'] = schools_gs['School'].apply(
    lambda x: x.split('/10')[0]
)
schools_gs['Rating'] = pd.to_numeric(schools_gs['Rating'], errors = 'coerce')
schools_gs['Name'] = schools_gs['School'].apply(lambda x: x.split('\n')[1])

schools_gs = schools_gs.dropna(subset=['Rating'])
schools_gs = schools_gs.drop(
    ['School'],
    axis = 1
)
schools_gs.head()

,Students,District,Rating,Name
0,808.0,Weston School District,10.0,Weston High School
1,361.0,West Hartford School District,3.0,Webster Hill School
2,331.0,New Haven School District,4.0,Sound School
3,603.0,Stamford School District,4.0,Julia A. Stark School
4,484.0,Danbury School District,7.0,Hayestown Avenue School


In [42]:
district_ratings = schools_gs[['Name', 'District','Students']].merge(
    (schools_gs['Students'] * schools_gs['Rating']).rename('Weight'),
    left_index=True,
    right_index=True
)
district_ratings.head()

,Name,District,Students,Weight
0,Weston High School,Weston School District,808.0,8080.0
1,Webster Hill School,West Hartford School District,361.0,1083.0
2,Sound School,New Haven School District,331.0,1324.0
3,Julia A. Stark School,Stamford School District,603.0,2412.0
4,Hayestown Avenue School,Danbury School District,484.0,3388.0


In [43]:
district_ratings = district_ratings.groupby('District').sum()

district_ratings = district_ratings[['Name']].merge(
    (district_ratings['Weight'] / district_ratings['Students']).rename('Rating').round(1),
    left_index=True,
    right_index=True
)['Rating']

district_ratings.head()

District
Achievement First Hartford Ac    6.0
Amistad Academy District         5.0
Andover School District          7.0
Ansonia School District          3.0
Area Cooperative Educational     4.1
Name: Rating, dtype: float64

In [44]:
districts_gs = pd.read_csv('districts_gs.csv')
print(districts_gs.shape)
districts_gs.head()

(205, 19)


,Unnamed: 0,district,district-href,english,math,ap_math,ap_sci,ap_other,low_income_perc,students_per_teacher,teachers_3yr,teacher_salary,rev_per_student,spend_per_student,spend_instruction,students,college,ap_total,local_rev
0,0,North Branford School District,https://www.greatschools.org/connecticut/north...,51%,51%,2%,7%,17%,16%,11.0,92%,"$64,421","$21,162","$17,939",49%,"1,777",86%,15%,63%
1,1,Trailblazers Academy District,https://www.greatschools.org/connecticut/stamf...,21%,21%,NaN,NaN,NaN,97%,NaN,NaN,NaN,"$24,464","$35,072",22%,119,21%,NaN,50%
2,2,Connecticut Technical High School,https://www.greatschools.org/connecticut/middl...,23%,23%,1%,NaN,NaN,38%,12.0,98%,NaN,NaN,NaN,NaN,"10,729",45%,<1%,NaN
3,3,New Beginnings Inc.,https://www.greatschools.org/connecticut/bridg...,22%,22%,NaN,NaN,NaN,91%,15.0,91%,NaN,"$13,834","$15,301",46%,497,22%,NaN,14%
4,4,Woodstock School District,https://www.greatschools.org/connecticut/woods...,46%,46%,NaN,NaN,NaN,10%,14.0,98%,"$66,339","$23,542","$15,388",42%,850,46%,NaN,60%


In [45]:
districts_acs = pd.read_csv('districts_acs.csv')
print(districts_acs.shape)
districts_acs.head()

(166, 3)


,name,households,income
0,Andover School District,1216,105328
1,Ashford School District,1774,70952
2,Barkhamsted School District,1336,109688
3,Bethany School District,2018,129133
4,Bozrah School District,986,90156


In [46]:
districts_merged = pd.merge(
    districts_gs,
    districts_acs,
    how = 'inner',
    left_on = 'district',
    right_on = 'name'
).drop(
    ['district'],
    axis = 1
)
districts_merged

,Unnamed: 0,district-href,english,math,ap_math,ap_sci,ap_other,low_income_perc,students_per_teacher,teachers_3yr,...,rev_per_student,spend_per_student,spend_instruction,students,college,ap_total,local_rev,name,households,income
0,0,https://www.greatschools.org/connecticut/north...,51%,51%,2%,7%,17%,16%,11.0,92%,...,"$21,162","$17,939",49%,"1,777",86%,15%,63%,North Branford School District,5339,90461
1,4,https://www.greatschools.org/connecticut/woods...,46%,46%,NaN,NaN,NaN,10%,14.0,98%,...,"$23,542","$15,388",42%,850,46%,NaN,60%,Woodstock School District,3249,86821
2,5,https://www.greatschools.org/connecticut/union...,69%,69%,NaN,NaN,NaN,NaN,10.0,100%,...,"$30,675","$21,962",44%,53,69%,NaN,80%,Union School District,374,87083
3,7,https://www.greatschools.org/connecticut/stamf...,48%,43%,5%,10%,24%,51%,12.0,94%,...,"$22,121","$20,039",56%,"15,816",71%,28%,76%,Stamford School District,49141,93059
4,8,https://www.greatschools.org/connecticut/new-b...,20%,11%,2%,5%,13%,75%,14.0,80%,...,"$18,528","$16,601",59%,"10,002",48%,13%,17%,New Britain School District,28232,46499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,199,https://www.greatschools.org/connecticut/berli...,65%,62%,3%,9%,19%,15%,11.0,89%,...,"$23,725","$18,691",46%,"2,781",85%,30%,64%,Berlin School District,8181,101127
162,200,https://www.greatschools.org/connecticut/monro...,73%,73%,19%,NaN,NaN,3%,11.0,98%,...,"$27,624","$23,217",63%,893,81%,26%,85%,Region 09,6251,136658
163,201,https://www.greatschools.org/connecticut/canaa...,70%,44%,NaN,NaN,NaN,15%,7.0,90%,...,"$37,013","$35,171",56%,71,44%,NaN,79%,Canaan School District,507,80298
164,203,https://www.greatschools.org/connecticut/westo...,75%,75%,14%,17%,28%,2%,12.0,98%,...,"$26,975","$23,320",49%,"2,329",93%,41%,85%,Weston School District,3447,222535
